<a href="https://colab.research.google.com/github/NavSanya/MyRelationalDatabase/blob/main/NavSanyaAnand_Project_6878418392.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Game Plan
##### IDEA:
###### SELECT:
* Get all = SELECT *
* Get  ___ = SELECT ___
###### FROM:
* From table ___ = FROM ___
###### WHERE:
* Where ___ = WHERE ___
* greater than = >
* less than = <
* at least = greater than equal to = greater than or equal to = >=
* at most = less than equal to = less than or equal to = <=
* equals = equal to = =
###### JOIN:
* combine tables __ where __ = JOIN ___ on ___
###### GROUP BY:
* find sum/avg/... of column ___ of table ___ =   SELECT column1, aggregate_function(column2) FROM table_name GROUP BY column1;
###### HAVING:
* where ___ = HAVING ____
###### ORDER BY:
* ___ in asc/desc order = ORDER BY ___ ASC/DESC
###### INSERT INTO:
* Add [values] to table ___ in columns [names] = INSERT INTO table_name (column1, column2, column3, ...) VALUES (value1, value2, value3, ...)
###### CREATE TABLE:
* create a table ___ with the following column and datatypes: [col1: datatype1, col2: datatype2 ...]
###### UPDATE:
* update ___ in table ___ with ___ = UPDATE table_name SET column1 = value1, column2 = value2, ... WHERE condition;

In [1]:
import re
import csv

In [4]:
# Function to copy selected columns from source CSV to a new CSV
def copyColumns(sourceCsvFile, destinationCsvFile, columnsToCopy):
    try:
        with open(sourceCsvFile, 'r', newline='', encoding='utf-8') as source_file:
            reader = csv.DictReader(source_file)
            source_headers = reader.fieldnames

            # Check if all columns to copy exist in the source CSV
            for column in columnsToCopy:
                if column not in source_headers:
                    print(f"Column '{column}' not found in the source CSV.")
                    return

            with open(destinationCsvFile, 'w', newline='', encoding='utf-8') as dest_file:
                writer = csv.DictWriter(dest_file, fieldnames=columnsToCopy)
                writer.writeheader()

                for row in reader:
                    # Extract the desired columns
                    selected_data = {col: str(row[col]) for col in columnsToCopy}
                    writer.writerow(selected_data)


        print("Selected columns copied to", destinationCsvFile)
        convert_csv_types(destinationCsvFile)

    except FileNotFoundError:
        print("Source CSV file not found.")
    except Exception as e:
        print("An error occurred:", str(e))

def convert_csv_types(fileName):
    try:
        with open(fileName, mode='r', newline='', encoding='utf-8') as input_csv_file:

            reader = csv.reader(input_csv_file)

            updatedData = []
            for row in reader:
                converted_row = []
                for item in row:
                    # Try to convert to int or float; if not possible, keep as is
                    try:
                        converted_item = int(item)
                    except ValueError:
                        try:
                            converted_item = float(item)
                        except ValueError:
                            converted_item = item  # Keep it as a string
                    converted_row.append(converted_item)
                updatedData.append(converted_row)
        with open(fileName, mode='w', newline='', encoding='utf-8') as output_csv_file:
            writer = csv.writer(output_csv_file)
            writer.writerows(updatedData)

        print(f"Data types converted and saved to {fileName}")

    except FileNotFoundError:
        print(f"File '{fileName}' not found.")
    except Exception as e:
        print("An error occurred:", str(e))

# loading all the tables
def createIndivisualTables():
    # Essentials Table
    copyColumns("IMDBTop250Movies.csv", "essentials.csv",["rank", "name", "year"])
    copyColumns("IMDBTop250Movies.csv", "viewerDeets.csv",["rank", "rating", "genre", "certificate", "run_time", "tagline"])
    copyColumns("IMDBTop250Movies.csv", "mediaDeets.csv",["rank", "budget", "tagline", "box_office"])
    copyColumns("IMDBTop250Movies.csv", "makers.csv",["rank", "casts", "directors", "writers"])

def LoadCsvFile(fileName):
            table = []
            with open(fileName, mode='r', newline='',  encoding='utf-8') as file:
                reader = csv.reader(file)
                header = next(reader, None)
                if header:
                    for row in reader:
                        data = {header[i]: value for i, value in enumerate(row)}
                        table.append(data)
                else:
                    table = list(reader)
            file.close()
            return table


In [5]:
createIndivisualTables()

Selected columns copied to essentials.csv
Data types converted and saved to essentials.csv
Selected columns copied to viewerDeets.csv
Data types converted and saved to viewerDeets.csv
Selected columns copied to mediaDeets.csv
Data types converted and saved to mediaDeets.csv
Selected columns copied to makers.csv
Data types converted and saved to makers.csv


In [ ]:
#actual db model functions - NOT TO RUN
class Database:
        def __init__(self):
                self.tables = []

        #CREATE TABLE
        def create_table(self, table_name, columns):
            self.tables[table_name] = {"columns": columns, "data": []}

        #INSERT INTO
        def insert_into(self, table_name, values):
            if table_name not in self.tables:
                raise ValueError("Table not found")
            if len(values) != len(self.tables[table_name]["columns"]):
                raise ValueError("Number of values does not match number of columns")
            self.tables[table_name]["data"].append(values)

        #SELECT FROM WHERE
        def get(self, table_name, condition):
            if table_name not in self.tables:
                raise ValueError("Table not found")
            results = []
            for row in self.tables[table_name]["data"]:
                if condition(row):
                    results.append(row)
            return results

        def display_table(self, table_name):
            if table_name not in self.tables:
                raise ValueError("Table not found")
            print(f"Table: {table_name}")
            print("Columns:", self.tables[table_name]["columns"])
            for row in self.tables[table_name]["data"]:
                print(row)

        # JOIN
        def join(self, table1_name, table2_name, common_column):
            if table1_name not in self.tables or table2_name not in self.tables:
                raise ValueError("Tables not found")

            table1 = self.tables[table1_name]
            table2 = self.tables[table2_name]

            if common_column not in table1["columns"] or common_column not in table2["columns"]:
                raise ValueError("Common column not found in both tables")

            results = []
            common_column_index1 = table1["columns"].index(common_column)
            common_column_index2 = table2["columns"].index(common_column)

            for row1 in table1["data"]:
                for row2 in table2["data"]:
                    if row1[common_column_index1] == row2[common_column_index2]:
                        results.append(row1 + row2)

            return results

        #GROUP BY
        def group_by(self, table_name, group_column):
            if table_name not in self.tables:
                raise ValueError("Table not found")

            if group_column not in self.tables[table_name]["columns"]:
                raise ValueError("Grouping column not found in the table")

            grouped_data = {}
            group_column_index = self.tables[table_name]["columns"].index(group_column)

            for row in self.tables[table_name]["data"]:
                key = row[group_column_index]
                if key not in grouped_data:
                    grouped_data[key] = []
                grouped_data[key].append(row)

            return grouped_data

In [6]:
class DatabaseFunctions:
        #actual db model functions
        def LoadCsvFile(self, fileName):
            table = []
            with open(fileName, mode='r', newline='',  encoding='utf-8') as file:
                reader = csv.reader(file)
                header = next(reader, None)
                if header:
                    for row in reader:
                        data = {header[i]: value for i, value in enumerate(row)}
                        table.append(data)
                else:
                    table = list(reader)
            file.close()
            return table

        #CREATE TABLE
        def createTable(self, table_name, columns):
            fileName = str(table_name) + ".csv"
            try:
                with open(fileName, mode='w', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow(columns)
                    print(f"Table \'{table_name}\' created.")
            except FileNotFoundError:
                print(f"File '{fileName}' not found")
            except Exception as e:
                print("An error occurred:", str(e))

        #INSERT INTO
        def insertInto(self, table_name, columns, values):
            fileName = str(table_name) + ".csv"
            try:
                with open(fileName, mode='r', newline='', encoding='utf-8') as fileRead:
                    reader = csv.reader(fileRead)
                    currentData = [row for row in reader]

                for row in currentData:
                    for colName, item in zip(columns, values):
                        colIndex = currentData[0].index(colName)
                        row[colIndex] = item

                with open(fileName, mode='a', newline='', encoding='utf-8') as fileWrite:
                    writer = csv.writer(fileWrite)
                    convertRow = []
                    for item in values:
                        try:
                            converted_item = int(item)
                        except ValueError:
                            try:
                                converted_item = float(item)
                            except ValueError:
                                converted_item = item  # Keep it as a string
                        convertRow.append(converted_item)

                    writer.writerow(convertRow)
                    print(f"Data types converted and saved to {table_name}")

            except FileNotFoundError:
                print(f"Table '{table_name}' not found.")
            except Exception as e:
                print("An error occurred:", str(e))

        #GET FROM WHERE
        def get(self, table_name, columns, condition):
            fileName = f"{table_name}.csv"
            try:
                selectedData = []
                with open(fileName, mode='r', newline='', encoding='utf-8') as input_csv_file:
                    reader = csv.reader(input_csv_file)
                    header = next(reader)  # Read the header row

                    data_types = []  # List to store inferred data types for each column
                    row1 = next(reader)
                    for column in row1:
                        if column.isdigit():
                            if column.find('$') != 0:
                                column = column.replace("$","")
                            data_types.append(int)
                        elif column.replace(".", "", 1).isdigit():
                            data_types.append(float)
                        else:
                            data_types.append(str)

                    # Read and convert data rows based on inferred data types
                    for row in reader:
                        converted_row = []
                        for value, data_type in zip(row, data_types):
                            if data_type == int:
                                if value.find('$') != 0:
                                    value = value.replace("$",'')
                                converted_row.append(int(value,20))
                            elif data_type == float:
                                converted_row.append(float(value))
                            else:
                                converted_row.append(value)
                        if columns == [None]:
                            columns = header
                        # Evaluate the condition and select rows that match
                        if condition:
                            if eval(condition, {}, dict(zip(header, converted_row))):
                                selected_row = {column: value for column, value in zip(header, converted_row) if column in columns}
                                selectedData.append(selected_row)
                return selectedData

            except FileNotFoundError:
                print(f"Table '{table_name}' not found.")
            except Exception as e:
                print("An error occurred:", str(e))


        def displayTable(self, table_name):
            fileName = str(table_name) + ".csv"
            data = self.LoadCsvFile(fileName)
            data = data[:10]
            print(*data, sep="\n")

        # JOIN
        def join(self, table1_name, table2_name, common_column):
            #still need to do
            return

        #GROUP BY
        def group_by(self, table_name, group_column):
            #still need to do
            return

In [8]:
#test dbfunctions

# # load csv - DONE
dbfunc = DatabaseFunctions()
# list = islice(dbfunc.LoadCsvFile("makers.csv"), 10)
# print(*list, sep="\n")

# #create table - DONE
# dbfunc.createTable(table_name="testTable", columns=["a","b","c"])

# # insert into - DONE
# dbfunc.insertInto("testTable", ["a","b", "c"], ["10", "2.5", "Hello"])

# # display Table - DONE
# dbfunc.displayTable("makers")

# get function - TO DO
res = dbfunc.get("makers", ["rank", "directors"], "rank >= 90 and rank < 100")
# print(res)
print(*res, sep="\n")

{'rank': '1', 'casts': 'Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,Clancy Brown,Gil Bellows,Mark Rolston,James Whitmore,Jeffrey DeMunn,Larry Brandenburg,Neil Giuntoli,Brian Libby,David Proval,Joseph Ragno,Jude Ciccolella,Paul McCrane,Renee Blaine,Scott Mann', 'directors': 'Frank Darabont', 'writers': 'Stephen King,Frank Darabont'}
{'rank': '2', 'casts': 'Marlon Brando,Al Pacino,James Caan,Diane Keaton,Richard S. Castellano,Robert Duvall,Sterling Hayden,John Marley,Richard Conte,Al Lettieri,Abe Vigoda,Talia Shire,Gianni Russo,John Cazale,Rudy Bond,Al Martino,Morgana King,Lenny Montana', 'directors': 'Francis Ford Coppola', 'writers': 'Mario Puzo,Francis Ford Coppola'}
{'rank': '3', 'casts': 'Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,Maggie Gyllenhaal,Gary Oldman,Morgan Freeman,Monique Gabriela Curnen,Ron Dean,Cillian Murphy,Chin Han,Nestor Carbonell,Eric Roberts,Ritchie Coster,Anthony Michael Hall,Keith Szarabajka,Colin McFarlane,Joshua Harto', 'directors': 'Chri

In [9]:
class QueryLanguageProcessing:
    def __init__(self):
        self.inputQuery = ""
        self.variables = []



    def getCondition(self, strCondition):
        # Define mapping of words to symbols
        word_to_symbol = {
            'is greater than': '>',
            'is less than': '<',
            'is at least': '>=',
            'is greater than or equal to': '>=',
            'is at most': '<=',
            'is less than or equal to': '>=',
            'is equal to': '==',
            'and': 'and',
            'or': 'or'
        }

        # Replace words with corresponding symbols
        for word, symbol in word_to_symbol.items():
            strCondition = strCondition.replace(word, symbol)

        # Replace 'x' and 'y' with their respective variable names
        strCondition = re.sub(r'\bx\b', 'x', strCondition)
        strCondition = re.sub(r'\by\b', 'y', strCondition)

        return strCondition

    def processStatement(self, statement):

        # regular expressions to match different parts of the statement
        getPattern = r"GET (.+) FROM TABLE (\w+) (?:WHERE (.+))?"
        getAllPattern = r"GET ALL FROM TABLE (\w+)(?: WHERE (.+))?"
        combineTablesPattern = r"COMBINE TABLES (.+) WHERE (.+)"
        findAggrigateFuncPattern = r"FIND (?:SUM|AVG|...)  OF COLUMN/S (.+) FROM TABLE (.+?) GROUP BY (.+)"
        orderPattern = r"IN (.+) (ASC|DESC) ORDER"
        AddValuesIntoPattern = r"ADD VALUES \((.+)\) INTO (.+) \((.+)\)"
        CreateTablePattern = r"CREATE A TABLE (.+) WITH THE FOLLOWING COLUMNS: (.+)"
        updatePattern = r"UPDATE (.+) IN TABLE (.+) WITH (.+)"
        showTable = r"SHOW TABLE (.+)"

        dbFuncObj = DatabaseFunctions()

        #get all statement
        if re.match(getAllPattern, statement):
            match = re.match(getAllPattern, statement)
            # columns = match.group(1).split(',')
            table = match.group(1)
            if match.group(2) is not None:
                whereConditionStr = match.group(2)
            else:
                whereConditionStr = "True"
            whereCondition = self.getCondition(whereConditionStr)
            # Process the SELECT statement here
            result = dbFuncObj.get(table_name=table, columns=[None], condition=whereCondition)
            print(result)
            return result

        #get colums statement
        elif re.match(getPattern, statement):
            match = re.match(getPattern, statement)
            columns = match.group(1).split(',')
            table = match.group(2)
            if match.group(3) is not None:
                whereConditionStr = match.group(3)
            else:
                whereConditionStr = "True"
            whereCondition = self.getCondition(whereConditionStr)
            print(f"Table Name = {table}, column name = {columns}, condition = {whereCondition}")
            # Process the SELECT statement here
            result = dbFuncObj.get(table_name=table, columns=columns, condition=whereCondition)
            return result

        elif re.match(CreateTablePattern, statement):
            match = re.match(CreateTablePattern, statement)
            table = match.group(1)
            columns = match.group(2).split(",")
            # Process the CREATE TABLE statement here
            dbFuncObj.createTable(table,columns)
            return "Done"


In [10]:

# Test Functions


queryObj = QueryLanguageProcessing()

# #getCondition
# x = 10
# y = 20
# english_sentence = "True"
# conditional_statement = queryObj.getCondition(english_sentence)
# print("English Sentence:", english_sentence)
# print("Conditional Statement:", conditional_statement)
# print(eval(conditional_statement))

# #get statement
# statement = "GET rank,writers FROM TABLE makers WHERE rank is greater than 20 and rank is less than 25"
# result = queryObj.processStatement(statement)
# print(*result, sep="\n")

#get all statement
statement = "GET ALL FROM TABLE mediaDeets WHERE budget is at most box_office"
result = queryObj.processStatement(statement)
print(*result, sep="\n")

# # create Table
# statement = "CREATE A TABLE testTableQuery WITH THE FOLLOWING COLUMNS: a,b"
# result = queryObj.processStatement(statement)
# print(*result, sep="\n")

An error occurred: invalid literal for int() with base 20: '$3300000'
None


TypeError: ignored

### Game Plan
* Each Indivisual Table is converted to a CSV File
###  DB Details
##### * Columns in original db
* rank - int
* name - string
* year - int
* rating - float
* genre - string/list
* certificate - String(Can make an enumerator)
* run_time - hours and minutes
* tagline - String
* budget - int (Long)
* box_office - int
* casts - list string
* directors - list string
* writers - list strings

### Tables we make
#### * Movie Essentials
* rank(PK), name, year
#### * Movie Details for viewers
* rank(FK), rating, genre, tagline, certificate
#### * Movie details for media
* rank(FK), budget, box_office
#### * Movie Makers
* rank(FK), casts, directors, writers